# Performance Benchmarking

**Yuning Zhang, TU Delft, 2021.10.27**

In [10]:
using Random

### Allocate a large sequence of random locations 

In [ ]:
function _rand_loc_spherical(r_max=1.0::Real, r_min=0.0::Real; projection=:false)
    r=rand()*(r_max-r_min)+r_min
    ϕ=rand()*2pi
    if projection
        θ=pi/2
    else
        θ=rand()*pi
    end
    [r*sin(θ)*cos(ϕ),r*sin(θ)*sin(ϕ),r*cos(θ)]
end

In [ ]:
@time collect(_rand_loc_spherical() for i in 1:10^6);

  0.122624 seconds (1.04 M allocations: 116.910 MiB, 19.80% compilation time)


In [21]:
function rand_locs_spherical(r_max=1.0::Real, r_min=0.0::Real; N=1::Int, projection=:false)
    M=zeros(N,3)
    r=rand(N).*(r_max-r_min).+r_min
    ϕ=rand(N).*2pi
    if projection
        θ=ones(N).*pi/2
    else
        θ=rand(N).*pi
    end
    collect(zip(r.*sin.(θ).*cos.(ϕ),r.*sin.(θ).*sin.(ϕ),r.*cos.(θ)))
end

rand_locs_spherical (generic function with 3 methods)

In [31]:
M=@time rand_locs_spherical(N=10^6);

  0.072342 seconds (22 allocations: 114.442 MiB, 2.49% gc time)


In [32]:
function rand_locs_spherical(r_max=1.0::Real, r_min=0.0::Real; N=1::Int, projection=:false)
    M=zeros(N,3)
    r=rand(N).*(r_max-r_min).+r_min
    ϕ=rand(N).*2pi
    if projection
        θ=ones(N).*pi/2
    else
        θ=rand(N).*pi
    end
    
    M[:,1]=r.*sin.(θ).*cos.(ϕ)
    M[:,2]=r.*sin.(θ).*sin.(ϕ)
    M[:,3]=r.*cos.(θ)

    M
end

rand_locs_spherical (generic function with 3 methods)

In [36]:
M=@time rand_locs_spherical(N=10^6);

  0.064265 seconds (20 allocations: 91.553 MiB, 1.57% gc time)


In [37]:
typeof(M)

Matrix{Float64} (alias for Array{Float64, 2})

**Conclusion1, it's better to use vectorized approach**

In [6]:
function rand_locs_cubic(a::Real, b::Real; N=1, projection=:false)
    M=zeros(N,3)
    if projection
        for i in 1:N
            M[i,:]=rand([-1,1],3).*rand(3).*(b-a).+a
        end
    else
        for i in 1:N
            M[i,1:2]=rand([-1,1],2).*rand(2).*(b-a).+a
        end
    end
end

rand_locs_cubic (generic function with 1 method)

In [7]:
@time rand_locs_cubic(1,0,N=10^6,projection=:true)

  0.237117 seconds (4.00 M allocations: 434.891 MiB, 29.62% gc time)


In [8]:
@time rand_locs_cubic(1,0,N=10^6,projection=:false)

  0.240182 seconds (4.00 M allocations: 389.099 MiB, 30.89% gc time)


In [21]:
function rand_locs_cubic(a::Real, b::Real; N=1, projection=:false)
    M=zeros(N,3)
    rand!(M,[1,-1])
    M.*=rand(N,3).*(b-a).+a
    if projection
        M[:,end].=0.0
    end
    M
end

rand_locs_cubic (generic function with 1 method)

In [35]:
@time rand_locs_cubic(1,0,N=10^5);

  0.001701 seconds (5 allocations: 4.578 MiB)


In [34]:
@time rand_locs_cubic(1,0,N=10^6,projection=:true);

  0.028170 seconds (5 allocations: 45.777 MiB, 4.26% gc time)


**Conclusion2, Use less allocations to reduce garbage collection time**

In [68]:
rand_locs(N::Int,dim::Int,a::Real)=a*[rand(Float64,dim).-1/2 for i in 1:N]

rand_locs (generic function with 1 method)

In [69]:
M=@time rand_locs(10^6,3,2);

  0.243390 seconds (3.00 M allocations: 335.694 MiB, 36.95% gc time)


In [70]:
typeof(M)

Vector{Vector{Float64}} (alias for Array{Array{Float64, 1}, 1})

In [75]:
function rand_locs(N::Int,dim::Int,a::Real)
    M=zeros(N,dim)
    rand!(zeros(N,dim))
    a*M.-1/2
end

rand_locs (generic function with 1 method)

In [76]:
@time rand_locs(10^6,3,2);

  0.028047 seconds (8 allocations: 91.553 MiB)


In [77]:
function rand_locs(N::Int,dim::Int,a::Real)
    a*rand!(zeros(N,dim)).-1/2
end

rand_locs (generic function with 1 method)

In [78]:
@time rand_locs(10^6,3,2);

  0.019772 seconds (6 allocations: 68.665 MiB)


In [79]:
typeof(M)

Vector{Vector{Float64}} (alias for Array{Array{Float64, 1}, 1})

**Conclusion 3. Generate a matrix is more efficnent compared with Array{Array{Float64}}. Use eachrow as generator.**

In [83]:
function rand_locs_cubic(a::Real, b::Real; N=1::Int, dim=3)
    M=zeros(N,3)
    rand!(@view(M[:,1:dim]),[1,-1])
    @view(M[:,1]).*=rand(N).*(b-a).+a
    if dim>1
        @view(M[:,2:dim]).*=rand(N,dim-1).*b
        for i in 1:N
            shuffle!(@view M[i,1:dim])
        end
    end
    M
end

rand_locs_cubic (generic function with 1 method)

In [85]:
@time rand_locs_cubic(2,1,N=10^6,dim=2);

  0.036026 seconds (7 allocations: 38.147 MiB)


## Test Performane of the ensemble FID